In [20]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm

import os
os.environ["R_HOME"] = r"C:\Program Files\R\R-4.0.2"
os.environ["PATH"]   = r"C:\Program Files\R\R-4.0.2\bin\x64" + ";" + os.environ["PATH"]

from pymer4.models import Lm, Lmer
from itertools import combinations
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn import preprocessing
from sklearn.model_selection import StratifiedKFold

seed = 667

In [3]:
# ignore convergence warnings
import warnings
from statsmodels.tools.sm_exceptions import ConvergenceWarning
warnings.simplefilter('ignore', ConvergenceWarning) 
warnings.simplefilter('ignore', RuntimeWarning) 
warnings.simplefilter('ignore', UserWarning) 

In [4]:
df = pd.read_csv('dementia.tsv', sep='\t')
df = df.rename(columns={'Subject ID': 'ID'})

In [5]:
min_max_scaler = preprocessing.MinMaxScaler()
df['eTIV'] = min_max_scaler.fit_transform(df[['eTIV']])

In [6]:
df

,ID,Group,Visit,Sex,Age,EDUC,SES,MMSE,CDR,eTIV,nWBV,Demented,Group_num,CDR_int
0,1,Nondemented,1,1,87,14,2.0,27.0,0.0,0.981069,0.696,0,0,0
1,1,Nondemented,2,1,88,14,2.0,30.0,0.0,1.000000,0.681,0,0,0
2,4,Nondemented,1,0,88,18,3.0,28.0,0.0,0.121381,0.710,0,0,0
3,4,Nondemented,2,0,90,18,3.0,27.0,0.0,0.104677,0.718,0,0,0
4,5,Nondemented,1,1,80,12,4.0,28.0,0.0,0.649220,0.712,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
349,185,Demented,2,1,82,16,1.0,28.0,0.5,0.653675,0.694,1,1,1
350,185,Demented,3,1,86,16,1.0,26.0,0.5,0.648107,0.675,1,1,1
351,186,Nondemented,1,0,61,13,2.0,30.0,0.0,0.237194,0.801,0,0,0
352,186,Nondemented,2,0,63,13,2.0,30.0,0.0,0.246102,0.796,0,0,0


In [7]:
# Split data into train and test sets
np.random.seed(667)
train_mask = np.random.rand(df.shape[0]) < .2

test_df = df[~train_mask]
train_df = df[train_mask]
test_df = sm.add_constant(test_df)

print('Training data:')
print('total observations: ' + str(train_df.shape[0]))
print(df.head())
print('Testing data:')
print('total observations: ' + str(test_df.shape[0]))
print(test_df.head())

Training data:
total observations: 65
   ID        Group  Visit  Sex  Age  EDUC  SES  MMSE  CDR      eTIV   nWBV  \
0   1  Nondemented      1    1   87    14  2.0  27.0  0.0  0.981069  0.696   
1   1  Nondemented      2    1   88    14  2.0  30.0  0.0  1.000000  0.681   
2   4  Nondemented      1    0   88    18  3.0  28.0  0.0  0.121381  0.710   
3   4  Nondemented      2    0   90    18  3.0  27.0  0.0  0.104677  0.718   
4   5  Nondemented      1    1   80    12  4.0  28.0  0.0  0.649220  0.712   

   Demented  Group_num  CDR_int  
0         0          0        0  
1         0          0        0  
2         0          0        0  
3         0          0        0  
4         0          0        0  
Testing data:
total observations: 289
   const  ID        Group  Visit  Sex  Age  EDUC  SES  MMSE  CDR      eTIV  \
0    1.0   1  Nondemented      1    1   87    14  2.0  27.0  0.0  0.981069   
2    1.0   4  Nondemented      1    0   88    18  3.0  28.0  0.0  0.121381   
3    1.0   4  Non

In [8]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)

In [9]:
# For brute-force search, we need to get all sublists from the list of columns except the empty sublist
def sublists(columns):
    sublists = []
    
    for i in range(0, len(columns)+1):
        temp = [list(x) for x in combinations(columns, i)]
        if len(temp)>0:
            sublists.extend(temp)
            
    sublists.remove([])
            
    return sublists 

# Here we create a brute-force search function which runs 5-fold cross-validation for each sublist
# We can define which objective function to use for optimization
def brute_search(data, response, r_vars, covariates, ident, penalty, obj_fn):
    resp = data[response]
    preds = data[covariates]
    preds = sm.add_constant(preds)
    preds_r = data[r_vars]
    search_space = sublists(preds.columns.tolist())
    
    bestobj = -np.inf
    best_cols = None
    
    for cols in search_space:
        objs = []
        for train_index, test_index in skf.split(preds[cols], resp):
            res = sm.BinomialBayesMixedGLM(resp.iloc[train_index], preds.iloc[train_index][covariates], preds_r.iloc[train_index][random_vars], 
                                           ident).fit_vb()
#             tmpobj = res.llf - penalty*len(cols)
            test_preds = res.predict(preds.iloc[test_index][cols])
            objs.append(obj_fn(resp.iloc[test_index], test_preds))
        tmpobj = sum(objs) / len(objs)
#         print('{0}: {1}'.format(cols, tmpobj))
#         print(objs)
        if tmpobj > bestobj:
            bestobj = tmpobj
            best_cols = cols
            
    res = sm.BinomialBayesMixedGLM(resp.iloc[test_index], preds.iloc[test_index][best_cols], preds_r.iloc[test_index][random_vars], 
                                  ident).fit_vb()
    return bestobj, best_cols, res

# GLMM for `Demented` response
## `Subject` random variable

In [162]:
covariates = ['Age', 'Sex', 'EDUC', 'SES', 'eTIV', 'nWBV', 'MMSE', 'Visit']
random_vars = ['ID']

In [163]:
res = sm.BinomialBayesMixedGLM.from_formula('Demented ~ MMSE + nWBV + SES + EDUC + Age + Sex + eTIV', random, train_df).fit_vb()
print(res.summary())

               Binomial Mixed GLM Results
           Type Post. Mean Post. SD   SD  SD (LB) SD (UB)
---------------------------------------------------------
Intercept     M     2.1136   0.3340                      
MMSE          M    -0.5195   0.0120                      
nWBV          M     1.5449   0.4515                      
SES           M     0.4459   0.1303                      
EDUC          M     0.0398   0.0223                      
Age           M     0.1140   0.0043                      
Sex           M     0.9578   0.5285                      
eTIV          M    -0.9086   0.6983                      
Subject ID    V    -0.9994   0.9994 0.368   0.050   2.717
Visit         V    -0.6934   0.7873 0.500   0.104   2.414
Parameter types are mean structure (M) and variance
structure (V)
Variance parameters are modeled as log standard
deviations


In [164]:
res = sm.BinomialBayesMixedGLM(train_df.Demented, sm.add_constant(train_df[covariates]), train_df[random_vars], [0]).fit_vb()
print(res.summary())

             Binomial Mixed GLM Results
      Type Post. Mean Post. SD   SD  SD (LB) SD (UB)
----------------------------------------------------
const    M     2.1087   0.3349                      
Age      M     0.1142   0.0043                      
Sex      M     0.9695   0.5296                      
EDUC     M     0.0397   0.0223                      
SES      M     0.4459   0.1307                      
eTIV     M    -0.9208   0.6998                      
nWBV     M     1.5403   0.4526                      
MMSE     M    -0.5230   0.0121                      
Visit    M     0.0698   0.1755                      
VC_1     V    -0.9994   0.9994 0.368   0.050   2.717
Parameter types are mean structure (M) and variance
structure (V)
Variance parameters are modeled as log standard
deviations


In [165]:
preds = res.predict(sm.add_constant(test_df[covariates + random_vars]))

In [166]:
roc_auc_score(test_df.Demented, preds)

0.5117079889807162

In [167]:
accuracy_score(test_df.Demented, [0 if i < 0.5 else 1 for i in preds])

0.5813148788927336

### Model selection

In [168]:
def demented_obj_fn(gt, preds):
    return accuracy_score(gt, [0 if i < 0.5 else 1 for i in preds])

In [173]:
print('Brute-force')
bf_obj, bf_cols, bf_res = brute_search(train_df, 'Demented', random_vars, covariates, [0], 1, demented_obj_fn)
print(bf_res.summary())
preds = bf_res.predict(test_df[bf_cols])
loglike_test = (test_df.Demented*np.log(preds) + (1-test_df.Demented)*np.log(1-preds)).sum()
print('llt: {0}:'.format(loglike_test))
print('auc: {0}:'.format(roc_auc_score(test_df.Demented, preds)))
print('acc: {0}:'.format(accuracy_score(test_df.Demented, [0 if i < 0.5 else 1 for i in preds])))
print()

Brute-force
             Binomial Mixed GLM Results
      Type Post. Mean Post. SD   SD  SD (LB) SD (UB)
----------------------------------------------------
Age      M     0.2938   0.0139                      
Sex      M     3.2992   1.2758                      
EDUC     M     0.8442   0.0711                      
SES      M     1.6896   0.4337                      
eTIV     M    -0.3771   1.4544                      
nWBV     M     0.2948   1.2087                      
MMSE     M    -1.5057   0.0398                      
Visit    M    -1.0502   0.5960                      
VC_1     V    -0.9805   0.9811 0.375   0.053   2.669
Parameter types are mean structure (M) and variance
structure (V)
Variance parameters are modeled as log standard
deviations


AttributeError: 'BayesMixedGLMResults' object has no attribute 'aic'

In [175]:
print('llt: {0}:'.format(loglike_test))
print('auc: {0}:'.format(roc_auc_score(test_df.Demented, preds)))
print('acc: {0}:'.format(accuracy_score(test_df.Demented, [0 if i < 0.5 else 1 for i in preds])))
print(bf_cols)
print()

llt: -292.46369707433803:
auc: 0.8074085005903188:
acc: 0.7716262975778547:
['Age', 'Sex', 'EDUC', 'SES', 'eTIV', 'nWBV', 'MMSE', 'Visit']



## `Visit` random variable

In [176]:
covariates = ['Age', 'Sex', 'EDUC', 'SES', 'eTIV', 'nWBV', 'MMSE']
random_vars = ['Visit']

In [177]:
random = {'Visit': '0 + Visit'}
res = sm.BinomialBayesMixedGLM.from_formula('Demented ~ MMSE + nWBV + SES + EDUC + Age + Sex + eTIV', random, df).fit_vb()
print(res.summary())

               Binomial Mixed GLM Results
          Type Post. Mean Post. SD   SD  SD (LB) SD (UB)
--------------------------------------------------------
Intercept    M     7.0799   0.1361                      
MMSE         M    -0.5205   0.0048                      
nWBV         M     2.7947   0.1855                      
SES          M     0.2142   0.0501                      
EDUC         M     0.0378   0.0090                      
Age          M     0.0480   0.0017                      
Sex          M     1.5127   0.1971                      
eTIV         M    -1.1552   0.2795                      
Visit        V    -0.8871   0.9032 0.412   0.068   2.508
Parameter types are mean structure (M) and variance
structure (V)
Variance parameters are modeled as log standard
deviations


In [178]:
res = sm.BinomialBayesMixedGLM(train_df.Demented, sm.add_constant(train_df[covariates]), train_df[random_vars], [0]).fit_vb()
print(res.summary())

             Binomial Mixed GLM Results
      Type Post. Mean Post. SD   SD  SD (LB) SD (UB)
----------------------------------------------------
const    M     2.1263   0.3313                      
Age      M     0.1112   0.0043                      
Sex      M     0.9493   0.5242                      
EDUC     M     0.0462   0.0221                      
SES      M     0.4306   0.1291                      
eTIV     M    -0.9126   0.6917                      
nWBV     M     1.5452   0.4479                      
MMSE     M    -0.5085   0.0119                      
VC_1     V    -0.6958   0.7885 0.499   0.103   2.414
Parameter types are mean structure (M) and variance
structure (V)
Variance parameters are modeled as log standard
deviations


In [179]:
preds = res.predict(sm.add_constant(test_df[covariates + random_vars]))

In [180]:
roc_auc_score(test_df.Demented, preds)

0.8112947658402204

In [181]:
accuracy_score(test_df.Demented, [0 if i < 0.5 else 1 for i in preds])

0.740484429065744

### Model Selection

In [182]:
print('Brute-force')
bf_obj, bf_cols, bf_res = brute_search(train_df, 'Demented', random_vars, covariates, [0], 1, demented_obj_fn)
print(bf_res.summary())
preds = bf_res.predict(test_df[bf_cols])
loglike_test = (test_df.Demented*np.log(preds) + (1-test_df.Demented)*np.log(1-preds)).sum()
print('llt: {0}:'.format(loglike_test))
print('auc: {0}:'.format(roc_auc_score(test_df.Demented, preds)))
print('acc: {0}:'.format(accuracy_score(test_df.Demented, [0 if i < 0.5 else 1 for i in preds])))
print()

Brute-force
            Binomial Mixed GLM Results
     Type Post. Mean Post. SD   SD  SD (LB) SD (UB)
---------------------------------------------------
Age     M     0.2207   0.0130                      
Sex     M     3.0543   1.1994                      
EDUC    M     0.9608   0.0666                      
SES     M     1.4482   0.4100                      
eTIV    M    -0.4582   1.3599                      
nWBV    M     0.3170   1.1604                      
MMSE    M    -1.3378   0.0371                      
VC_1    V    -0.3484   0.6589 0.706   0.189   2.636
Parameter types are mean structure (M) and variance
structure (V)
Variance parameters are modeled as log standard
deviations
llt: -232.1260251433474:
auc: 0.7975698543880363:
acc: 0.7681660899653979:



## `Subject` and `Visit` random variable

In [183]:
covariates = ['Age', 'Sex', 'EDUC', 'SES', 'eTIV', 'nWBV', 'MMSE']
random_vars = ['Visit', 'ID']

In [184]:
random = {"Subject ID": '0 + ID', 'Visit': '0 + Visit'}
res = sm.BinomialBayesMixedGLM.from_formula('Demented ~ MMSE + nWBV + SES + EDUC + Age + Sex + eTIV', random, df).fit_vb()
print(res.summary())

               Binomial Mixed GLM Results
           Type Post. Mean Post. SD   SD  SD (LB) SD (UB)
---------------------------------------------------------
Intercept     M     6.8377   0.1377                      
MMSE          M    -0.5465   0.0049                      
nWBV          M     2.6648   0.1877                      
SES           M     0.2247   0.0507                      
EDUC          M     0.0235   0.0092                      
Age           M     0.0563   0.0018                      
Sex           M     1.5465   0.1984                      
eTIV          M    -1.2227   0.2828                      
Subject ID    V    -0.9976   0.9976 0.369   0.050   2.712
Visit         V    -0.8857   0.9022 0.412   0.068   2.506
Parameter types are mean structure (M) and variance
structure (V)
Variance parameters are modeled as log standard
deviations


In [185]:
res = sm.BinomialBayesMixedGLM(train_df.Demented, sm.add_constant(train_df[covariates]), train_df[random_vars], [0, 0]).fit_vb()
print(res.summary())

             Binomial Mixed GLM Results
      Type Post. Mean Post. SD   SD  SD (LB) SD (UB)
----------------------------------------------------
const    M     2.1138   0.3333                      
Age      M     0.1136   0.0043                      
Sex      M     0.9520   0.5276                      
EDUC     M     0.0397   0.0222                      
SES      M     0.4448   0.1299                      
eTIV     M    -0.9035   0.6970                      
nWBV     M     1.5452   0.4505                      
MMSE     M    -0.5171   0.0120                      
VC_1     V    -1.4225   0.6812 0.241   0.062   0.942
Parameter types are mean structure (M) and variance
structure (V)
Variance parameters are modeled as log standard
deviations


In [186]:
preds = res.predict(sm.add_constant(test_df[covariates + random_vars]))

In [187]:
roc_auc_score(test_df.Demented, preds)

0.790633608815427

In [188]:
accuracy_score(test_df.Demented, [0 if i < 0.5 else 1 for i in preds])

0.7370242214532872

### Model Selection

In [190]:
print('Brute-force')
bf_obj, bf_cols, bf_res = brute_search(train_df, 'Demented', random_vars, covariates, [0, 0], 1, demented_obj_fn)
print(bf_res.summary())
preds = bf_res.predict(test_df[bf_cols])
loglike_test = (test_df.Demented*np.log(preds) + (1-test_df.Demented)*np.log(1-preds)).sum()
print('llt: {0}:'.format(loglike_test))
print('auc: {0}:'.format(roc_auc_score(test_df.Demented, preds)))
print('acc: {0}:'.format(accuracy_score(test_df.Demented, [0 if i < 0.5 else 1 for i in preds])))
print()

Brute-force
            Binomial Mixed GLM Results
     Type Post. Mean Post. SD   SD  SD (LB) SD (UB)
---------------------------------------------------
Age     M     0.2704   0.0135                      
Sex     M     3.2193   1.2596                      
EDUC    M     0.8405   0.0685                      
SES     M     1.5741   0.4313                      
eTIV    M    -0.3847   1.4381                      
nWBV    M     0.2744   1.1856                      
MMSE    M    -1.4873   0.0386                      
VC_1    V    -1.1086   0.5994 0.330   0.100   1.095
Parameter types are mean structure (M) and variance
structure (V)
Variance parameters are modeled as log standard
deviations
llt: -273.8659862082908:
auc: 0.8096221959858324:
acc: 0.7577854671280276:

